In [1]:
import torch
import torch.nn.functional as F
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [3]:
model = GPT2LMHeadModel.from_pretrained('openai-community/gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Задание 1:

Вам даны attention-скоры для последовательности ниже. Необходимо определить позицию токена, на которую больше всего "обращал внимание" токен на позиции 10 во второй attention head на первом слое модели *(вся упомянутая нумерация идёт с нуля)* . Ответом должен быть номер позиции:

In [ ]:
model_input = tokenizer('Границы ключ переломлен', return_tensors='pt')

In [ ]:
model_output = model(**model_input, output_attentions=True)

In [ ]:
attentions = model_output["attentions"]  # Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length)

В attentions лежат веса attention для каждого слоя

In [ ]:
model_input['input_ids']

tensor([[  140,   241, 21169, 16142, 22177, 18849,   141,   228, 45035, 12466,
           118, 30143,   141,   236,   141,   229, 12466,   123, 16843, 21169,
         16843, 30143, 25443,   120, 30143, 16843, 22177]])

In [ ]:
for i in range(len(model_input['input_ids'][0])):
  print(tokenizer.decode(model_input['input_ids'][0][i]))

�
�
р
а
н
и
�
�
ы
 �
�
л
�
�
�
�
 �
�
е
р
е
л
о�
�
л
е
н


In [ ]:
import numpy as np

In [ ]:
torch.argmax(attentions[1][0][2][10])

tensor(9)

# Задание 2:

С заданной моделью model и входными данными model_input сгенерируйте продолжение входных данных с помощью beam search. Генерируем при следующих параметрах:
- не более 30 новых токенов
- рассматриваем топ-2 beam-а, затем из них выбирается наиболее вероятный
- early_stopping=True, no_repeat_ngram_size=2

Ответом должна быть строка(оригинальная строка плюс всё, что вы нагенерировали), включая кавычки!
Подробнее о генерации и её параметрах можно почитать здесь: \
https://huggingface.co/blog/how-to-generate

In [ ]:
out = model.generate(**model_input, max_new_tokens=30, num_beams=2, early_stopping=True, no_repeat_ngram_size=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.decode(out[0])

'Границы ключ переломленный строшить на это изгор'

# Задание 3:

Даны логиты модели для следующего токена и порог p=0.95 для nucleus sampling. Посчитайте минимальное кол-во токенов, сумма вероятностей которых даёт не менее p. Ответом должно быть целое число:

In [ ]:
last_token_logits = model_output.logits[0][-1]
p = 0.95

In [ ]:
sorted_indices = torch.argsort(last_token_logits, descending=True)
logits_sorted = last_token_logits[sorted_indices]
probabilities = torch.softmax(logits_sorted, dim=-1)
print(probabilities)
res = 0
i = 0
while res <= p:
  res += probabilities[i]
  i += 1
print(i - 1)

tensor([5.1043e-01, 1.5689e-01, 3.7870e-02,  ..., 1.3559e-12, 2.2233e-13,
        1.5157e-13], grad_fn=<SoftmaxBackward0>)
15


# Задание 4:

На лекции вам рассказывали про токены, которые "ломают" GPT-модели, т.е. вызывают у них неадекватное поведение. Давайте и мы попробуем найти такие токены для модели GPT2! \
Как их нашли в эксперименте SolidGoldMagiKarp: такие токены находились ближе всего к "центру масс" эмбеддингов модели, т.е. к усреднённому эмбеддингу по всем имеющимся эмбеддингам модели. Ответом на данное задание должен быть 14-ый(начиная с нуля) токен среди всех, отсортированных по убыванию расстояния от "центра масс":

In [5]:
token_embeddings = model.transformer.wte.weight

In [6]:
token_embeddings

Parameter containing:
tensor([[-0.1101, -0.0393,  0.0331,  ..., -0.1364,  0.0151,  0.0453],
        [ 0.0403, -0.0486,  0.0462,  ...,  0.0861,  0.0025,  0.0432],
        [-0.1275,  0.0479,  0.1841,  ...,  0.0899, -0.1297, -0.0879],
        ...,
        [-0.0445, -0.0548,  0.0123,  ...,  0.1044,  0.0978, -0.0695],
        [ 0.1860,  0.0167,  0.0461,  ..., -0.0963,  0.0785, -0.0225],
        [ 0.0514, -0.0277,  0.0499,  ...,  0.0070,  0.1552,  0.1207]],
       requires_grad=True)

In [7]:
center_of_mass = torch.mean(token_embeddings, dim=0)
distances = torch.norm(token_embeddings - center_of_mass, dim=-1)
index_of_token = torch.argsort(distances, descending = True)[14]
token_text = tokenizer.decode(index_of_token.item())
token_text

'ngth'

# Задание 5

Даны логиты модели для следующего токена и температура $\tau = 1.5$. Убедимся в том, что распределение вероятностей на токенах станет более равномерным при применении $\tau$, т.е. у токенов с небольшой вероятностью ДО применения температуры  эта вероятность увеличится при $\tau > 1$. Примените при подсчёте софтмакса указанное $\tau$ и посмотрите, как увеличится вероятность токена под номером 228(отсчёт от нуля) по сравнению с вероятностью этого же токена, но БЕЗ применения температуры. Ответом должно быть число- указанная разность * 1e5 с точностью до 4 знаков  

In [ ]:
last_token_logits = model_output.logits[0][-1]
tau = 1.5

In [ ]:
prob_no_temp = torch.softmax(last_token_logits, dim=-1)

In [ ]:
prob_no_temp[228]

tensor(3.1464e-07, grad_fn=<SelectBackward0>)

In [ ]:
logits_with_temp = last_token_logits / tau
prob_with_temp = torch.softmax(logits_with_temp, dim=-1)


In [ ]:
prob_with_temp[228]

tensor(1.4765e-05, grad_fn=<SelectBackward0>)

In [ ]:
print((prob_with_temp[228] - prob_no_temp[228]) * 1e5)

tensor(1.4450, grad_fn=<MulBackward0>)
